# QnA System

## Install & Library

In [1]:
# !nvidia-smi

In [2]:
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm, tqdm_notebook

from KoBERT.kobert.utils import get_tokenizer
from KoBERT.kobert.pytorch_kobert import get_pytorch_kobert_model

from konlpy.tag import Mecab

from sklearn.metrics.pairwise import cosine_similarity

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

import warnings
warnings.filterwarnings(action='ignore')

pd.set_option('max_row', None)
pd.set_option('display.max_colwidth', -1)

np.set_printoptions(formatter={'float_kind': lambda x: "{0:0.5f}".format(x)})

device = 'cuda:1' if torch.cuda.is_available() else 'cpu'
device

'cuda:1'

## Path

In [3]:
model_path = './Model/'
qna_path = '../Data/QnA/'
review_path = '../Data/Review/Review_embedding_SN_new/'
data_path = '../Data/'

## BERT Model

In [4]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i]) for i in dataset] # sent_idx를 제거
        
    def __getitem__(self, i):
        return (self.sentences[i])

    def __len__(self):
        return (len(self.sentences))

In [5]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,  ##### 수정
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        
        classifier = self.classifier(pooler)
        
        if self.dr_rate:
            out = self.dropout(classifier)
  
        return pooler, out

## Parameter

In [6]:
## Setting parameters
max_len = 210 #수정
batch_size = 16 # 64 
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [7]:
# load model
bertmodel, vocab = get_pytorch_kobert_model()
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model
using cached model
using cached model


In [8]:
# load pre-trained KoBERTClassifier
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
model.load_state_dict(torch.load(model_path+'type1_KoBERT_new.pt'))
model.eval() 

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

## Mecab

In [9]:
# load mecab
mecab = Mecab()

In [10]:
def make_question_mecab_tokens(question):
    # mecab 돌리기
    que_mecab = mecab.pos(question[0])

    # 조사 제거
    morpheme = ['NNG','NNP','NNB','NNBC','NR','NP','VV','VA','VX','VCP','VCN','MM','MAG','MAJ','IC','SN'] # SN 추가
    tmp = []
    que_tokens = []

    for t in que_mecab:
        if t[1] in morpheme:
            que_tokens.append(t[0])

    if len(que_tokens)==0:
        que_tokens.append('')

    # 한 문장으로 결합
    que_tokens_str = [' '.join(que_tokens)]
    
    return que_tokens_str

## Input Embedding & Classification

In [11]:
def input_BERTClassifier(model, test_dataloader):
    for batch_id, (token_ids, valid_length, segment_ids) in enumerate(test_dataloader):

        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length  
        
        test_embedding, test_probability = model.forward(token_ids, valid_length, segment_ids)

        test_embedding = np.array(test_embedding.tolist())
        test_probability = np.array(test_probability.tolist())

        torch.cuda.empty_cache() # GPU 캐시 삭제
        
    # class 
    test_class = np.argmax(np.array(test_probability))
        
    return test_embedding, test_class

## Semantic Search

In [12]:
def Semantic_Search_Question(question, input_embedding, input_class, product_id):
    # load file
    df = pd.read_pickle(qna_path+'question_embedding_SN_new.pkl')
    
    # 같은 상품 내의 질문만 추출
    df = df[df['product_id'] == product_id]
    df.reset_index(drop=True, inplace=True)

    # input_class와 같은 label만 추출
    label_idx = np.array(np.where(np.array(df['question_class'].tolist()) == input_class)).tolist()[0]
    df_in_input = df.loc[label_idx]
    df_in_input.reset_index(drop=True, inplace=True)
    
    # 질문 embedding
    question_embedding = df_in_input['question_embedding']

    # make tensor
    corpus_embeddings = torch.tensor(question_embedding)
    query_embedding = torch.tensor(input_embedding)
    
    # cosine similarity 계산
    cos_scores = cosine_similarity(query_embedding, corpus_embeddings)
    cos_socres_round = np.array(list(map(lambda x:int(x*1000)/1000, cos_scores[0]))) 
    df_in_input['input_question'] = np.repeat(question, len(df_in_input))
    df_in_input['score'] = cos_socres_round
    
    # 0.95 이상만 반환
    df_in_input_top = df_in_input[df_in_input['score']>=0.95]
    df_in_input_top.sort_values(by='score', ascending=False, inplace=True)
    df_in_input_top.reset_index(drop=True, inplace=True)
    
    return df_in_input_top

In [13]:
def Semantic_Search_Review(question, input_embedding, input_class, product_id):
    # load review data
    df = pd.read_pickle(review_path+'review_'+str(product_id)+'_embedding.pkl')
    
    # input_class와 같은 label만 추출
    label_idx = np.array(np.where(np.array(df['review_class'].tolist()) == input_class)).tolist()[0]
    
    if len(label_idx) != 0: # 같은 label의 리뷰가 하나라도 있으면
        df_in_input = df.loc[label_idx]
        df_in_input['using_label'] = input_class
        df_in_input.reset_index(drop=True, inplace=True)    
    else: # 하나도 없으면
        return pd.DataFrame() # 빈 데이터 프레임 반환
    
    # 리뷰 embedding
    review_embedding = df_in_input['review_embedding']
              
    # make tensor
    corpus_embeddings = torch.tensor(review_embedding)
    query_embedding = torch.tensor(input_embedding)
    
    # cosine similarity 계산
    cos_scores = cosine_similarity(query_embedding, corpus_embeddings)
    cos_socres_round = np.array(list(map(lambda x:int(x*1000)/1000, cos_scores[0]))) 
    df_in_input.loc[:,'input_question'] = np.repeat(question, len(df_in_input))
    df_in_input['score'] = cos_socres_round
    
    # 0.95 이상만
    df_in_input_top = df_in_input[df_in_input['score']>=0.95]
    df_in_input_top.sort_values(by='score', ascending=False, inplace=True)
    df_in_input_top.reset_index(drop=True, inplace=True)
    
    return df_in_input_top

## Custom

### Sentiment Analysis

In [14]:
def Text_Sentiment_Review(df):
    # sentiment table 불러오기
    with open(data_path+'table.pickle', 'rb') as file:
        table = pickle.load(file)
 
    # 초기화
    df['neg'] = 0
    df['neut'] = 0
    df['pos'] = 0

    # df 리뷰 전체 리스트
    review_list = df['comment_mecab'].apply(lambda x:x.replace('\n','').split(' ')).values.tolist()

    # table에 있는 경우만 남기기
    review_list = [[word for word in review if word in table] for review in review_list]

    # 리스트 내 word를 table 내 숫자로 바꾸기
    neg_list = [[float(table[word]['Neg']) for word in review] for review in review_list]
    pos_list = [[float(table[word]['Pos']) for word in review] for review in review_list]

    # 값 sum()
    neg = [sum(c) for c in neg_list]
    pos = [sum(c) for c in pos_list]

    # df 문장 단어 개수
    df['count'] = df['comment_mecab'].apply(lambda x:len(x.replace('\n','').split(' ')))

    # df에 구한 값 추가 & 개수로 나누기
    df['neg'] = round(neg/df['count'], 3)
    df['pos'] = round(pos/df['count'], 3)

    # 부정 점수 0.3 이하 & 긍정 점수 0.3 이상인 경우만 추출
    df = df[(df['neg']<=0.3)&(df['pos']>=0.3)]
    
    # column drop
    df_return = df.drop(columns=['count'])

    return df_return

### Word Count

In [15]:
def word_count_in_wordcloud_Review(df):
    # wordcloud dict 불러오기   
    with open(data_path+'word_dict.pickle', 'rb') as file:
        word_dict = pickle.load(file)
    
    # 전체 단어 count 개수
    df['word_count'] = df['comment_mecab'].apply(lambda x: len(x.split(' ')))
    
    # 예측된 label의 wordcloud에 포함된 단어
    df['word_count_in_cloud'] = 0
    for i in range(len(df)):
        df['word_count_in_cloud'][i] = len([w for w in df['comment_mecab'][i].replace('\n','').split(' ') if w in(word_dict[df['review_class'][i]][:30])]) # 30개만 불러오기
    
    # column 제거
    df_return = df.drop(columns=['word_count_in_cloud'])
    
    return df_return

## Select Top

In [16]:
def select_top2_Question(df):
    # score 기준 내림차순으로 정렬
    df.sort_values(by=['score'], ascending=False, inplace=True)
    df.reset_index(drop=True, inplace=True) # reset index
        
    if df.shape[0] < 2: # 개수가 2개보다 적으면
        df_return = df
    else:
        df_return = df[:2]

    # 필요한 column만 반환
    df_return = df_return[['input_question', 'score', 'question', 'answer']]

    return df_return

In [17]:
def select_top5_Review(df, sentiment, wordcount):    
    # Custom 여부에 따라 정렬
    if (sentiment == False) and (wordcount == False):
        sort_list = ['score', 'praise_count', 'image']
        df.sort_values(by=sort_list, ascending=False, inplace=True)
        
    if (sentiment == True) and (wordcount == False):
        sort_list = ['score', 'pos', 'neg', 'praise_count', 'image']
        df.sort_values(by=sort_list, ascending=[False, False, True, False, False], inplace=True)
    
    if (sentiment == False) and (wordcount == True):
        sort_list = ['score', 'word_count', 'praise_count', 'image']
        df.sort_values(by=sort_list, ascending=False, inplace=True)
    
    if (sentiment == True) and (wordcount == True):
        sort_list = ['score', 'pos', 'neg', 'word_count', 'praise_count', 'image']
        df.sort_values(by=sort_list, ascending=[False, False, True, False, False, False], inplace=True)  
        
    # reset index
    df.reset_index(drop=True, inplace=True)
    
    if df.shape[0] < 5: # 5개보다 적으면
        df_return = df
    else:
        df_return = df[:5]
    
    # 필요한 column만 추출
    out_list = ['prod_id', 'prod_name', 'input_question', 'comment'] + sort_list
    df_return = df_return[out_list]
    
    return df_return

## Matching System

In [18]:
def matching_system(product_id, question, sentiment, wordcount):
    # run mecab
    que_mecab = make_question_mecab_tokens(question)

    # make dataloader
    input_data = BERTDataset(que_mecab, 0, tok, max_len, True, False)
    input_dataloader = torch.utils.data.DataLoader(input_data, batch_size=batch_size, num_workers=5)
    
    # input data embedding & class
    input_embedding, input_class = input_BERTClassifier(model, input_dataloader)
    
    
    # Semantic Search between input & question
    question_top = Semantic_Search_Question(question, input_embedding, input_class, product_id)
    # select qna top2
    question_top2 = select_top2_Question(question_top)
    
    
    # Semantic Search between input & review
    review_top = Semantic_Search_Review(question, input_embedding, input_class, product_id)
    # Make sentiment column
    if sentiment == True: 
        review_top = Text_Sentiment_Review(review_top)
    # Make word count column
    if wordcount == True:
        review_top = word_count_in_wordcloud_Review(review_top)
    # select review top5
    review_top5 = select_top5_Review(review_top, sentiment, wordcount)  

    return question_top2, review_top5

----

## Run Matching System

In [19]:
# custom 모델 list
tf_list = [['Base', False, False], ['Sentiment', True, False], ['Wordcount', False, True]]

# 매칭 시스템 실행 함수
def evaluation(id_, question_):
    df_display = pd.DataFrame(columns=['input_question'])
    for tf in tf_list:
        question, review = matching_system(id_, question_, 
                                           tf[1], # sentiment
                                           tf[2]) # wordcount      
        
        if tf[0] == 'Base':
            df_display = pd.concat([df_display, question[['input_question','question','answer']]])
            display(question[['input_question','question','answer']])
        print('이번 결과는: ', tf[0])
        df_display = pd.concat([df_display, review[['input_question','comment']]])
        display(review[['input_question','comment']])
    return df_display

In [20]:
# 예시
id_ = 388715
question_ = '나사 조립이 많이 어려운가요'

evaluation(id_, question_)

,input_question,question,answer
0,나사 조립이 많이 어려운가요,너무 쓰레기같아요 나사를끝까지돌려도 다 들어가지않고 다리가 덜렁거리고 나사 하나는 렌치로돌리는구멍이 막혀서 돌려넣을수도업고 아무리 저렴한제품이라도 이렇게 검수도 제대로안하고 판매하나요;; 고객센터는 전화도안받고 장난치는것도아니고 배송도 한오백년걸리는데 물건이따위로 보내주면 어떡하죠 ?.?,안녕하세요 고객님 먼데이하우스입니다.\r\n이용중 불편을 드려 죄송합니다.\r\n안타깝게도 구매하신 제품이 조립이 되어지면 반품이 불가하여 하자라고 생각되시는 부분의 사진을\r\n첨부하여 오늘의집 고객센터로 교환접수 해주시면 담당부서에서 교환여부 확인후 진행도와드리겠습니다.\r\n감사합니다.(2)
1,나사 조립이 많이 어려운가요,방금 주문했는데 하자 없이 꼼꼼하게 검수해서 보내주시면 감사하겠습니다! 그리고 혹시 유리가 깨져서 왔다는 후기가 있어서 .. 포장도 꼼꼼하게 해주세요 ㅠㅠ!!,안녕하세요 고객님~~먼데이하우스입니다.\r\n네 검수후 빠른 출고위해 최선을 다하겠습니다.\r\n구매해주셔서 감사드려요 ~~(1)


이번 결과는:  Base


,input_question,comment
0,나사 조립이 많이 어려운가요,아이고 주인님이 좋아하신다니 정말 다행입니다 감사합니다
1,나사 조립이 많이 어려운가요,스탠드 올려놓으려구 샀는데 너무 예뻐요! \n후기 보니 마감이 안좋다는 말들이 있어 걱정했는데 괜찮았구 \n배송은 좀 느려서 아쉬웠어요
2,나사 조립이 많이 어려운가요,배송 빨랐어요! 유리에 스크레치 있더라구요!ㅠ
3,나사 조립이 많이 어려운가요,베스트 제품인데 이유가 있겠죠?! 이뻐요~
4,나사 조립이 많이 어려운가요,조립도 간편하고 이뿌네요 저희 집 고영희 역시 저 협탁을 좋아합니다 ㅎ ㅎ


이번 결과는:  Sentiment


,input_question,comment
0,나사 조립이 많이 어려운가요,좋아요이건 그양쓸만함 굿굿굿구숰ㅋㅋㅋㅋ
1,나사 조립이 많이 어려운가요,조립도 쉽고 깔끔하고 예뻐요 오오오오
2,나사 조립이 많이 어려운가요,사진같은 느낌으로 깔끔하고 예쁩니다\n빠른 배송이 좋습니다
3,나사 조립이 많이 어려운가요,너무 좋아요 이쁘고 좋아요 굿굿 다시사료
4,나사 조립이 많이 어려운가요,마음에 들어요 생각보다 단단하더라구요


이번 결과는:  Wordcount


,input_question,comment
0,나사 조립이 많이 어려운가요,배송-휴일과 공휴일이 겹쳐서 4일걸림 포장 꼼꼼하게 해서 걱정했던 찍힘이나 오염 없었음 다만 위험하다고 유리판 테두리를 잘라서 줬는데 깨끗하게 잘린게 아니라서 더 위험해보임 조립-손힘이 약해서 나사 끼우는데 한참 땀뺌 마지막에 다리가 뭐가 잘못됐는지 균형이 안맞아서 한쪽 다리에 스티커 2개붙여줌 같이 오는 다리에붙이는 스티커가 있어서 이부분은 좋았음 조립 꼼꼼하게 안하면 나중에 유리끼울때 힘드니까 참고하셈 디자인-역시 이쁨 색도 내가 생각했던 색이라 마음에 듬 사실 실용적인것보단 디자인보고 산거라 딱히 불만없음 내구성-조립할때 유리끼우는부분 아귀가 안맞아서 한번 때렸더니 그부분이 그대로 부서짐. 아무래도 싼만큼 내구성이 그렇게 좋진 않은듯 근데 평소에 그정도 충격 줄일은 없으니까 4점줌 가격-총점 매겨보면 딱 괜찮은 가격인듯
1,나사 조립이 많이 어려운가요,배송은 무척 빨리 왔고 배송해 주시는 분도 친절해서 우선 좋았어요 다만 제품을 박스에서 열어 보니 유리 끝 부분에는 좀 깨져 있고 나무 2번째칸의 부분에 금이가서 깨져 있더라구요 ㅠㅠ 반품해야 하나 하다가 조립하면 보이지 않을 부분이고 무거운 물건을 올리지 않을 예정이라 바로 조립 했습니다 조립은 좀.... 뻑뻑 하지만 그게 더 고정 시켜 주는 부분이라 오히려 안심이 됬구요 색상은 화이트라 이뻐요 ㅎㅎ 같이구매한 꽃과도 잘 어울려서 맘에 쏙 들어요 아~~ 다만 구매 후 2틀 정도는 페인트 냄새가 좀 나서 밖에 두고 냄새를 뺀 후에 쓰면 좋을 것 같아요 ㅎㅎ 이쁜 제품을 구매해서 좋았습니다^^
2,나사 조립이 많이 어려운가요,"첫자취 첫 협탁이에요 워낙 이 디자인의 협탁이 유명해서 사는데 까지 힘들었어욬ㅋㅋㅋ 그래도 배송도 엄청빨리오고 생각했던데로 디자인도 이뻐요 다만 제 자취방에비해 너무 세련돼서 안어울리는감이 없지않아있지만?ㅋㅋㅋ 하지만 매우잘쓰고있답니닿ㅎㅎㅎㅎ 근데 저만 조립하는데 힘들었나요? 전 죽는줄알았어요,,,,그냥포기할뻔..ㅠ 저는 드라이버도없고해서 동봉된걸로 조였더니 손에물집잡히는줄..ㅠ 그래도 무사히 조립했습니다😂😂😂 아무튼 싸고이쁘게 잘샀어용 잘쓸겠습니당ㅎㅎㅎㅎ"
3,나사 조립이 많이 어려운가요,캔들거치대로 방하나 거실하나 총 두개 주문했습니다. 조립은 여자인제가 해도 쉬울정도였지만 나사로 조립할때 그분분에 옹이?같은거 땜에그런지 나무가 약해서 그런지 금이가더라구요 그래도 저렴한가격이라 그냥씁니다. 그리고 유리넣는부분 잘 넣어야 합니다 꽉들어가야 마지막조립때 잘 맞아떨어집니다. 디자인은 심플해서 좋습니다! 배송은엄청빠르네요 하루만에왔어요
4,나사 조립이 많이 어려운가요,잘샀어요!! 조립하는게 쫌 빡쎄긴했지만 어렵지않아요 손이 좀 아플뿐... 온수매트 넣어놓으려고 구매했어요 살짝 수평이 안맞는거같지만 상관없어유 나무에 얼룩이 묻어있긴하지만 자세히 안볼거니깐 상관읍구요 유리가 마감이 좀 별로던데 어차피 가려지니 상관없음! 저렴한데 생각보다 엄청 맘에 들어요 ㅎㅎ 열심히 꾸며볼게요~~~ 아참참 배송도 아주 완벽히 포장되어서 왔어요! 감사합니다!


,input_question,question,answer,comment
0,나사 조립이 많이 어려운가요,너무 쓰레기같아요 나사를끝까지돌려도 다 들어가지않고 다리가 덜렁거리고 나사 하나는 렌치로돌리는구멍이 막혀서 돌려넣을수도업고 아무리 저렴한제품이라도 이렇게 검수도 제대로안하고 판매하나요;; 고객센터는 전화도안받고 장난치는것도아니고 배송도 한오백년걸리는데 물건이따위로 보내주면 어떡하죠 ?.?,안녕하세요 고객님 먼데이하우스입니다.\r\n이용중 불편을 드려 죄송합니다.\r\n안타깝게도 구매하신 제품이 조립이 되어지면 반품이 불가하여 하자라고 생각되시는 부분의 사진을\r\n첨부하여 오늘의집 고객센터로 교환접수 해주시면 담당부서에서 교환여부 확인후 진행도와드리겠습니다.\r\n감사합니다.(2),NaN
1,나사 조립이 많이 어려운가요,방금 주문했는데 하자 없이 꼼꼼하게 검수해서 보내주시면 감사하겠습니다! 그리고 혹시 유리가 깨져서 왔다는 후기가 있어서 .. 포장도 꼼꼼하게 해주세요 ㅠㅠ!!,안녕하세요 고객님~~먼데이하우스입니다.\r\n네 검수후 빠른 출고위해 최선을 다하겠습니다.\r\n구매해주셔서 감사드려요 ~~(1),NaN
0,나사 조립이 많이 어려운가요,NaN,NaN,아이고 주인님이 좋아하신다니 정말 다행입니다 감사합니다
1,나사 조립이 많이 어려운가요,NaN,NaN,스탠드 올려놓으려구 샀는데 너무 예뻐요! \n후기 보니 마감이 안좋다는 말들이 있어 걱정했는데 괜찮았구 \n배송은 좀 느려서 아쉬웠어요
2,나사 조립이 많이 어려운가요,NaN,NaN,배송 빨랐어요! 유리에 스크레치 있더라구요!ㅠ
3,나사 조립이 많이 어려운가요,NaN,NaN,베스트 제품인데 이유가 있겠죠?! 이뻐요~
4,나사 조립이 많이 어려운가요,NaN,NaN,조립도 간편하고 이뿌네요 저희 집 고영희 역시 저 협탁을 좋아합니다 ㅎ ㅎ
0,나사 조립이 많이 어려운가요,NaN,NaN,좋아요이건 그양쓸만함 굿굿굿구숰ㅋㅋㅋㅋ
1,나사 조립이 많이 어려운가요,NaN,NaN,조립도 쉽고 깔끔하고 예뻐요 오오오오
2,나사 조립이 많이 어려운가요,NaN,NaN,사진같은 느낌으로 깔끔하고 예쁩니다\n빠른 배송이 좋습니다
